In [ ]:
# 메모리에 대화 기록을 '리스트'로 저장하는 클래스
from langchain_core.chat_history import InMemoryChatMessageHistory

# 모델을 생성할 때 대화 기록을 함께 전달. 래퍼wrapper 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory

# 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_openai import ChatOpenAI  

from langchain_core.messages import HumanMessage

In [ ]:
model = ChatOpenAI(model="gpt-4.1-mini")


store = {}  # 세션별 대화 기록을 저장할 딕셔너리
 
# 세션 ID에 따라 대화 기록을 가져오는 함수. RunnableWithMessageHistory 클래스 생성시 호출
def get_session_history(session_id: str):
        if session_id not in store:     # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
                store[session_id] = InMemoryChatMessageHistory()    # 메모리에 대화 기록을 저장할 객체 생성
        return store[session_id]        # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "abc2"}}   # 세션 ID 를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 angelica야.")],
    config=config,
 )

print(response.content)

안녕, Angelica! 만나서 반가워요. 어떻게 도와줄까요?


In [4]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
 )

print(response.content)

너의 이름은 Angelica야!


In [ ]:
config = {"configurable": {"session_id": "abc3"}}   # 세션 ID 를 변경하면 대화를 기억 못 함

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
 )

response.content

'죄송하지만 사용자의 이름을 알 수 없습니다. 알려주시면 기억하는 데 도움이 될 것 같아요!'

In [6]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
 )

response.content

'우리는 네 이름이 Angelica라는 얘기를 나눴어! 그 외에는 특별한 대화가 없었어. 더 이야기하고 싶은 게 있니?'

In [ ]:
config = {"configurable": {"session_id": "abc2"}}

# 스트림 방식으로 출력하기 : invoke() 메소드를 stream() 으로 변경
# 모델마다 설정된 토큰 크기 만큼 응답을 보내 줍니다.
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="|")

|Angel|ica|님|에| 대해| 주|어진| 정보|가| 적|어서| 정확|하게| 어느| 나라| 사람|인지| 맞|추|기는| 어렵|지만|,| 한국|어|로| 대|화|하고| 계|시|고| 이름|이| Angel|ica|이|신| 점|을| 고려|해|볼| 때|,| 한국|에| 거|주|하는| 외|국|인이|시|거나| 한국|어|를| 잘| 아|시는| 분|일| 수도| 있|겠|다고| 추|측|해|볼| 수| 있어|요|.

|만|약| 한국|과| 관련|된| 문화|에| 대해| 이야기|해|볼|까요|?|  
|예|를| 들어|,| 한국| 문화|는| 전|통|과| 현대|가| 조|화|롭게| 어|우|러|져| 있어|요|.| 한|복|이라는| 전|통| 의|상|과| K|-pop|,| 한|류| 드|라마|처럼| 세계|적으로| 인기| 있는| 현대| 문화|가| 공|존|하|죠|.| 또한|,| 한국|은| 예|절|을| 중|시|하고|,| 가족| 중심|의| 가치|관|이| 강|한| 사회|랍니다|.

|Angel|ica|님|이| 맞|다면| 한국| 문화|에| 대해| 더| 궁|금|한| 점| 있|나요|?| 아니|면| 다른| 나라| 문|화를| 원|하|시면| 알려|주세요|!||||